First we have to load in our data, stored as a .txt file. To protect private company info, the `file_path' variable is loaded separately.

In [1]:
import re
import pandas as pd
from datetime import datetime, timedelta

execfile("confidential.py")

# Open the file in read mode
with open(file_path, 'r') as file:
    # Read the contents of the file
    file_contents = file.read()

# Now you can use the `file_contents` variable which contains the content of the file
print(file_contents)


----------------------------------------------------------------------------------------------------
E2001  Check-In 05:14   Check-Out 13:50            1-Day                     SEP 2022
                                                                             +---------------------+
Day    Flt   Dep  Local   Arr  Local   Turn     Eqp   Block      Duty        | S  M  T  W  T  F  S |
 1     2295  EWR  06:14   MIA  09:18   001:07   32M   003:04                 |=====================|
 1     2594  MIA  10:25   EWR  13:35            32M   003:10                 |         -- -- -- 03 |
                                                                 008:36      |04 -- -- -- -- -- -- |
                                                    ----------------------   |-- -- -- -- -- -- -- |
Credit: 006:14                                        006:14     008:36      |-- -- -- -- -- -- -- |
TAFB: 008:36                                                                 |-- -- -- -- -- --    |
Crew 

Here we're concerned with extracting the flight table from the data for display.

In [2]:
# Extract the flight details using regular expressions
flight_data = re.findall(r'(\d)?\s+(\d+)\s+(\w{3})\s{2}(\d{2}:\d{2})\s{3}(\w{3})\s{2}(\d{2}:\d{2})\s*(\d{3}:\d{2})?\s*(\w{3})\s{3}(\d{3}:\d{2})', raw_data) # raw data is a simple dataset stored in confidential.py

columns = ['Day', 'Flt', 'Dep', 'Local_Dep', 'Arr', 'Local_Arr', 'Turn', 'Eqp', 'Block']
flight_table = pd.DataFrame(flight_data, columns=columns)
print(flight_table)

  Day   Flt  Dep Local_Dep  Arr Local_Arr    Turn  Eqp   Block
0   1  1015  EWR     09:00  PUJ     13:01  001:00  32M  004:01
1   1  1016  PUJ     14:01  EWR     18:04          32M  004:03
2   2  1229  EWR     07:30  CUN     10:26  001:00  32M  003:56
3   2  1230  CUN     11:26  EWR     16:20          32M  003:54
4   3  0573  EWR     07:00  TPA     09:46          32M  002:46
5   4  2274  TPA     08:50  EWR     11:31          32M  002:41


In [3]:
# Extract other variables of interest

## From the top line pull the pairing code and the number of days of the pairing

p_code = re.findall(r'[A-Z]\d+', raw_data)
p_code = p_code[0]
print(p_code)

n_days = re.findall(r'(\d+)-Day', raw_data)
n_days = int(n_days[0])
print(n_days)

## Near the bottom pull both the total credit and sum of the block times and use it to calculate the soft time. Plus record time away from base

credit_sum = re.findall(r'Credit:\s(\d{3}:\d{2})', raw_data)
credit_sum = credit_sum[0]
hours, minutes = map(int, credit_sum.split(':'))
credit_sum = timedelta(hours=hours, minutes=minutes)
print(credit_sum)

block_sum = re.findall(r'\w\s{40}(\d{3}:\d{2})', raw_data)
block_sum = block_sum[0]
hours, minutes = map(int, block_sum.split(':'))
block_sum = timedelta(hours=hours, minutes=minutes)
print(block_sum)

softtime = credit_sum - block_sum
print(softtime)

tafb = re.findall(r'TAFB:\s(\d{3}:\d{2})', raw_data)
tafb = tafb[0]
hours, minutes = map(int, tafb.split(':'))
tafb = timedelta(hours=hours, minutes=minutes)
print(tafb)

## Here, we concern ourselves with dates. We want the dates in the calendar on the right to match with the month and year above the calendar.

my = re.findall(r'[A-Z]{3}\s\d{4}', raw_data)
my = datetime.strptime(my[0],"%b %Y")
print(my)

d = list()
d_lines = re.findall(r'\|.*\|', raw_data)
for week in d_lines:
    d_strs = re.findall(r'\d+', week)
    for d_str in d_strs:
        d.append(d_str)

d = [int(num) for num in d]
mdy = [my + timedelta(days=item-1) for item in d]
for date in mdy:
    print(date)

## Further, we want check in and checkout times to be associated with these dates.

checkin = re.findall(r'Check-In\s(\d{2}:\d{2})', raw_data)
checkin = datetime.strptime(checkin[0],"%H:%M")
checkins = list()
for date in mdy:
    checkin_datetime = datetime.combine(date.date(), checkin.time())
    checkins.append(checkin_datetime)
print(checkins)

checkout = re.findall(r'Check-Out\s(\d{2}:\d{2})', raw_data)
checkout = datetime.strptime(checkout[0],"%H:%M")
checkouts = list()
for date in mdy:
    date = date + timedelta(days = n_days - 1)
    checkout_datetime = datetime.combine(date.date(), checkout.time())
    checkouts.append(checkout_datetime)
print(checkouts)

checkinouts = []
for item1, item2 in zip(checkins, checkouts):
    checkinouts.append((item1, item2))
print(checkinouts[1])



E2071
4
21:39:00
21:21:00
0:18:00
3 days, 3:46:00
2022-09-01 00:00:00
2022-09-10 00:00:00
2022-09-11 00:00:00
2022-09-17 00:00:00
2022-09-19 00:00:00
2022-09-20 00:00:00
2022-09-23 00:00:00
2022-09-27 00:00:00
[datetime.datetime(2022, 9, 10, 8, 0), datetime.datetime(2022, 9, 11, 8, 0), datetime.datetime(2022, 9, 17, 8, 0), datetime.datetime(2022, 9, 19, 8, 0), datetime.datetime(2022, 9, 20, 8, 0), datetime.datetime(2022, 9, 23, 8, 0), datetime.datetime(2022, 9, 27, 8, 0)]
[datetime.datetime(2022, 9, 13, 11, 46), datetime.datetime(2022, 9, 14, 11, 46), datetime.datetime(2022, 9, 20, 11, 46), datetime.datetime(2022, 9, 22, 11, 46), datetime.datetime(2022, 9, 23, 11, 46), datetime.datetime(2022, 9, 26, 11, 46), datetime.datetime(2022, 9, 30, 11, 46)]
(datetime.datetime(2022, 9, 11, 8, 0), datetime.datetime(2022, 9, 14, 11, 46))


In [5]:
#print(file_contents)

pairings = file_contents.split('----------------------------------------------------------------------------------------------------\n')
del pairings[0]
del pairings[-1]
#pairings_list = list()

table_columns = ['p_code', 'n_days', 'checkin', 'checkout', 'credit_sum', 'block_sum', 'softtime','flight_data']
table_data = pd.DataFrame(columns=table_columns)

for pairing in pairings:
    
    flight_data = re.findall(r'(\d)?\s+(\d+)\s+(\w{3})\s{2}(\d{2}:\d{2})\s{3}(\w{3})\s{2}(\d{2}:\d{2})\s*(\d{3}:\d{2})?\s*(\w{3})\s{3}(\d{3}:\d{2})', pairing) # raw data is a simple dataset stored in confidential.py

    columns = ['Day', 'Flt', 'Dep', 'Local_Dep', 'Arr', 'Local_Arr', 'Turn', 'Eqp', 'Block']
    flight_table = pd.DataFrame(flight_data, columns=columns)

    # Extract other variables of interest

    ## From the top line pull the pairing code and the number of days of the pairing

    p_code = re.findall(r'[A-Z]\d+', pairing)
    p_code = p_code[0]

    n_days = re.findall(r'(\d+)-Day', pairing)
    n_days = int(n_days[0])

    ## Near the bottom pull both the total credit and sum of the block times and use it to calculate the soft time. Plus record time away from base

    credit_sum = re.findall(r'Credit:\s(\d{3}:\d{2})', pairing)
    credit_sum = credit_sum[0]
    hours, minutes = map(int, credit_sum.split(':'))
    credit_sum = timedelta(hours=hours, minutes=minutes)

    block_sum = re.findall(r'\w\s{40}(\d{3}:\d{2})', pairing)
    block_sum = block_sum[0]
    hours, minutes = map(int, block_sum.split(':'))
    block_sum = timedelta(hours=hours, minutes=minutes)

    softtime = credit_sum - block_sum

    tafb = re.findall(r'TAFB:\s(\d{3}:\d{2})', pairing)
    tafb = tafb[0]
    hours, minutes = map(int, tafb.split(':'))
    tafb = timedelta(hours=hours, minutes=minutes)

    ## Here, we concern ourselves with dates. We want the dates in the calendar on the right to match with the month and year above the calendar.

    my = re.findall(r'[A-Z]{3}\s\d{4}', pairing)
    my = datetime.strptime(my[0],"%b %Y")

    d = list()
    d_lines = re.findall(r'\|.*\|', pairing)
    for week in d_lines:
        d_strs = re.findall(r'\d+', week)
        for d_str in d_strs:
            d.append(d_str)

    print(d_lines)

    d = [int(num) for num in d]
    mdy = [my + timedelta(days=item-1) for item in d]

    ## Further, we want check in and checkout times to be associated with these dates.

    checkin = re.findall(r'Check-In\s(\d{2}:\d{2})', pairing)
    checkin = datetime.strptime(checkin[0],"%H:%M")
    checkins = list()
    for date in mdy:
        checkin_datetime = datetime.combine(date.date(), checkin.time())
        checkins.append(checkin_datetime)

    checkout = re.findall(r'Check-Out\s(\d{2}:\d{2})', pairing)
    checkout = datetime.strptime(checkout[0],"%H:%M")
    checkouts = list()
    for date in mdy:
        date = date + timedelta(days = n_days - 1)
        checkout_datetime = datetime.combine(date.date(), checkout.time())
        checkouts.append(checkout_datetime)

    ## Now all that remains is to make our table_data with all pairings. But because we want to filter by dates, we actually make an observation for each date a pairing is offered as well.

    checkinouts = []
    for item1, item2 in zip(checkins, checkouts):
        checkinouts.append((item1, item2))
    
    for checkinout in checkinouts:
        checkin, checkout = checkinout
        obs = pd.DataFrame([[p_code, n_days, checkin, checkout, credit_sum, block_sum, softtime, flight_data]], columns = table_columns)
        table_data = pd.concat([table_data, obs], ignore_index=True)

print(table_data)    

#for pairing in pairings:
#print(pairings)

['| S  M  T  W  T  F  S |', '|=====================|', '|         -- -- -- 03 |', '|04 -- -- -- -- -- -- |', '|-- -- -- -- -- -- -- |', '|-- -- -- -- -- -- -- |', '|-- -- -- -- -- --    |']
['| S  M  T  W  T  F  S |', '|=====================|', '|         -- -- -- -- |', '|-- -- -- 07 08 09 -- |', '|11 12 -- 14 -- 16 -- |', '|18 19 20 21 22 23 -- |', '|25 26 27 -- 29 30    |']
['| S  M  T  W  T  F  S |', '|=====================|', '|         -- -- -- -- |', '|-- -- -- -- -- -- 10 |', '|-- -- -- -- -- -- 17 |', '|-- -- -- -- -- -- 24 |', '|-- -- -- -- -- --    |']
['| S  M  T  W  T  F  S |', '|=====================|', '|         31 01 02 -- |', '|-- -- 06 -- -- -- -- |', '|-- -- -- -- -- -- -- |', '|-- -- -- -- -- -- -- |', '|-- -- -- -- -- --    |']
['| S  M  T  W  T  F  S |', '|=====================|', '|         -- -- -- -- |', '|-- -- 06 -- -- -- -- |', '|-- -- -- -- -- -- -- |', '|-- -- -- -- -- -- -- |', '|-- -- -- -- -- --    |']
['| S  M  T  W  T  F  S |', '|====================